# Laboratorio 3
Modelo alternativo usando Random Forest.


Cargar librerias.

In [1]:
pip install torch torchvision

  Using cached torch-2.7.1-cp313-cp313-win_amd64.whl.metadata (28 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.7.1-cp313-cp313-win_amd64.whl (216.1 MB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 365.4 kB/s eta 0:00:04
   ------------ --------------------------- 0.5/1.7 MB 36


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install kagglehub scikit-learn

  Using cached scikit_learn-1.7.1-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached scipy-1.16.1-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.1-cp313-cp313-win_amd64.whl (8.7 MB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached scipy-1.16.1-cp313-cp313-win_amd64.whl (38.5 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

   ------------- -------------------------- 2/6 [scipy]
   ------------- -------------------------- 2/6 [scipy]
   ------------- -------------------------- 2/6 [scipy]
   ------------- -------------------------- 2/6 [scipy]
   ------------- -------------------------- 2/6 [scipy]
   ------------- -------------------------- 2/6 [scipy]
   ------------- ----------------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# 1. Carga del dataset


In [2]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformaciones (escalado, tamaño, conversión a tensor)
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),  # Normaliza a [0,1]
])


data_dir = "C:/Users/ricar/Downloads/PolyMNIST/MMNIST"

# Cargar conjuntos de entrenamiento y prueba
train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 5. Modelo con otro algoritmo: Random Forest

In [4]:
# --- 5. Modelo con otro algoritmo: Random Forest usando PyTorch

# Instalación si es necesario:
# !pip install torch torchvision scikit-learn pillow

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Parámetros y rutas
root_dir = r"C:/Users/ricar/Downloads/PolyMNIST/MMNIST"
subdirs = ['train', 'test']
labeldirs = ['m0', 'm1', 'm2', 'm3', 'm4']
batch_size = 128

# Transformación: escala a [0,1], tamaño 28x28 y canales reproducidos a 3
transform = transforms.Compose([
    transforms.Resize((28,28)),
    transforms.Lambda(lambda img: img.convert('RGB') if img.mode!='RGB' else img),
    transforms.ToTensor()
])

# Loader PyTorch para todas las modalidades juntas
# Usamos ImageFolder, por lo cual la estructura debe ser:
# root_dir/train/[m0,m1,...]/imagenes
# root_dir/test/[m0,m1,...]/imagenes

datasets_rf = {}
for split in subdirs:
    path = os.path.join(root_dir, split)
    ds = datasets.ImageFolder(path, transform=transform)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=(split=='train'))
    datasets_rf[split] = loader

# Función auxiliar: extraer datos y labels a numpy arrays (hasta un límite)
def extract_numpy(loader, max_samples=None):
    X_list, y_list = [], []
    count = 0
    for imgs, labels in loader:
        imgs = imgs.numpy()  # [B, C, H, W]
        B, C, H, W = imgs.shape
        imgs = imgs.transpose(0,2,3,1).reshape(B, -1)  # [B, H*W*C]
        X_list.append(imgs)
        y_list.append(labels.numpy())
        count += B
        if max_samples and count>=max_samples:
            break
    X = np.vstack(X_list)[:max_samples]
    y = np.hstack(y_list)[:max_samples]
    return X, y

# Extraer un subconjunto para entrenar y testear
X_train, y_train = extract_numpy(datasets_rf['train'], max_samples=10000)
X_test,  y_test  = extract_numpy(datasets_rf['test'],  max_samples=2000)

print(f"Tamaño X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Tamaño X_test:  {X_test.shape},  y_test:  {y_test.shape}")

# Entrenar Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Predecir y medir accuracy
y_pred = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred)
print(f"Random Forest - accuracy: {acc_rf:.4f}")


Tamaño X_train: (10000, 2352), y_train: (10000,)
Tamaño X_test:  (2000, 2352),  y_test:  (2000,)
Random Forest - accuracy: 0.8920


Exactitud del modelo: Se obtuvo un 89.2% de precisión con Random Forest. Esto significa que el modelo clasifica correctamente casi 9 de cada 10 dígitos.

Interpretación: Es un buen resultado para un modelo tradicional, considerando que PolyMNIST añade ruido visual.
Aun así, modelos basados en CNN suelen superar el 95% al aprovechar la estructura espacial de las imágenes.

Por lo tanto, Random Forest es una opción válida y eficiente si no se puede usar redes neuronales, aunque tiene un límite frente a modelos más complejos.